In [2]:
import pandas as pd
import numpy as np

In [3]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.metrics import fbeta_score, make_scorer
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score


# Reading Data

In [4]:
df = pd.read_csv('../../Dataset/NSL_new.csv')

In [5]:
df = df.drop( df[ (df.label != "normal") & (df.label != "neptune") & (df.label != "back") & (df.label != "land") & (df.label != "pod") & (df.label != "smurf") & (df.label != "teardrop") & (df.label != "teardrop") & (df.label != "mailbomb") & (df.label != "apache2") & (df.label != "processtable") & (df.label != "udpstorm") & (df.label != "worm")].index )
print(df.shape)

(113270, 123)


In [6]:
df = df.sample(n=20000)

In [7]:
df.head()

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,flag_RSTO,flag_RSTOS0,flag_RSTR,flag_S0,flag_S1,flag_S2,flag_S3,flag_SF,flag_SH,label
1988,0,337,578,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,normal
91552,0,316,367,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,normal
8063,28,46,46,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,normal
12345,0,324,4041,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,normal
121090,0,5,217,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,normal


In [8]:
X = df.drop(['label'], axis=1)
y = df['label']

In [9]:
scoring_metrics = {'accuracy' : make_scorer(accuracy_score), 
           'precision' : make_scorer(precision_score, average="micro"),
           'recall' : make_scorer(recall_score, average="micro"), 
           'f1_score' : make_scorer(f1_score, average="micro")}

In [10]:
y = y.replace( to_replace =  'normal' , value = 0 )

y = y.replace( to_replace =  'neptune' , value = 1 )
y = y.replace( to_replace =  'back' , value = 1 )
y = y.replace( to_replace =  'land' , value = 1 )
y = y.replace( to_replace =  'pod' , value = 1 )
y = y.replace( to_replace =  'smurf' , value = 1 )
y = y.replace( to_replace =  'teardrop' , value = 1 )
y = y.replace( to_replace =  'mailbomb' , value = 1 )
y = y.replace( to_replace =  'apache2' , value = 1 )
y = y.replace( to_replace =  'processtable' , value = 1 )
y = y.replace( to_replace =  'udpstorm' , value = 1 )
y = y.replace( to_replace =  'worm' , value = 1 )

In [11]:
y

1988      0
91552     0
8063      0
12345     0
121090    0
         ..
107220    1
51784     0
53071     1
92533     0
13264     1
Name: label, Length: 20000, dtype: int64

In [15]:
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neural_network import MLPClassifier

In [27]:
# Create Learners per layer
layer_one_estimators = [
                        ('knn_1', KNeighborsClassifier(n_neighbors=5, n_jobs=-1)),
                        ('rn_1', RandomForestClassifier(random_state=0, n_jobs=-1, n_estimators=90, max_depth=19, 
                                                        min_samples_split=0.1)),
                        ('ada_1', AdaBoostClassifier(random_state=0, learning_rate=0.1, n_estimators=1000))
                       ]

layer_two_estimators = [
                        ('dt_2', DecisionTreeClassifier(max_depth=13, min_samples_split=10)),
                        ('mlp_2', MLPClassifier(solver='adam', alpha=0.001, hidden_layer_sizes=20)),
                       ]

In [28]:
layer_two = StackingClassifier(estimators=layer_two_estimators, final_estimator=MLPClassifier())

In [29]:
# Create Final model by 
clf = StackingClassifier(estimators=layer_one_estimators, final_estimator=layer_two)

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)

In [34]:
value = clf.fit(X_train, y_train).score(X_test, y_test)

C:\Users\Mohmaed Ibrahim\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:568: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Mohmaed Ibrahim\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:568: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Mohmaed Ibrahim\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:568: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Mohmaed Ibrahim\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:568: ConvergenceWarning: Stochastic Optimizer: Maximum i

In [38]:
value

0.9736842105263158

In [50]:
clf_Score = pd.DataFrame(columns=['Accuracy'])
clf_Score.at[0, 'Accuracy'] = value
print(clf_Score)

   Accuracy
0  0.973684


In [ ]:
clf_Score.to